## modulok importálása

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

## weboldal tartalmának beolvasása

In [ ]:
url = 'https://en.wikipedia.org/wiki/Community_areas_in_Chicago'

# User-Agent azonosítóval enged a wikipedia, mindegy mit írunk
headers = {'User-Agent' : 'Mozilla/5.0'}

In [ ]:
# lehívjuk a wiki oldalt
response = requests.get(url = url, headers = headers)

# teljes weboldal tartalma
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
# prettify - szebben kiírja a beolvasott adatokat
# print(soup.prettify())

## táblázat kikeresése

In [ ]:
# minden táblázatot megkeres
soup.find_all("table")

In [ ]:
# csak a táblázatok class attribútumát írjuk ki: <table class = "xyz xyz">...</table>
for table in soup.find_all("table") :
	print(table.get('class'))

In [ ]:
# egyetlen tábla keresése
table = soup.find('table', class_ = 'wikitable sortable plainrowheaders mw-datatable')

In [ ]:
# soronként írjuk ki a táblát, amelyikben a parkok nevei vannak
# az első két sor header, az utolsó sor összegzés, ezeket nem vesszük figyelembe
for row in table.tbody.find_all('tr')[2 :-1] :
	# a harmadik sortól az utolsó előttiig van szükségnk az adatokra
	print(row.prettify()) # prettify olvashatóbbá teszi a kódot
	print('-' * 100)

In [ ]:
# soronként kiírjuk a cellák tartalmát - először a sorszámokat
for row in table.tbody.find_all('tr')[2 : -1] :
	cells = row.find_all('td')
	print(cells[0].get_text(strip = True))  # strip eltűnteti a felesleges whitespacet

In [ ]:
# soronként kiírjuk a cellák tartalmát - most a szöveges adatokat
for row in table.tbody.find_all('tr')[2 : -1] :
	h_cells = row.find_all('th')
	print(h_cells[0].get_text(strip = True))  # strip eltűnteti a felesleges whitespacet

In [ ]:
# soronként kiírjuk a cellák tartalmát - együtt
for row in table.tbody.find_all('tr')[2 : -1] :
	cells = row.find_all('td') # számok
	h_cells = row.find_all('th') # szövegek (headerek)
	print(cells[0].get_text(strip = True))  # strip eltűnteti a felesleges whitespacet
	print(h_cells[0].get_text(strip = True))
	print()

## list of dictionaries készítése a beolvasott adatokból

In [ ]:
data = []  # könyvtárak listája

for row in table.tbody.find_all('tr')[2 : -1] :
	cells = row.find_all('td')
	h_cells = row.find_all('th')

	# elmentjük változókba a megtisztított értékeket
	area_code = cells[0].get_text(strip = True)  # strip eltűnteti a felesleges whitespacet
	community_area_name = h_cells[0].get_text(strip = True)

	# a kiolvasott adatokat hozzáadjuk a listához kulcs-érték párként
	data.append({'area_code' : area_code, 'community_area' : community_area_name})

## dataframe készítése a list of dictionariesből

In [ ]:
community_areas_df = pd.DataFrame(data)
community_areas_df['area_code'] = community_areas_df['area_code'].astype('int')
# community_areas_df.head()

## kiíratás csv-be

In [ ]:
community_areas_df.to_csv('dim_community_areas.csv', index = False)